In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Installation\\spark-3.3.2-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = (
            SparkSession
                .builder
                .appName("SparkSQLApp")
                .master("local[4]")
    
                .config("spark.dynamicAllocation.enabled", "false")
                .config("spark.sql.adaptive.enabled", "false")
    
                .getOrCreate()
        )

sc = spark.sparkContext

spark

In [3]:
from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read Yellow Taxis data

In [4]:
# Create schema for Yellow Taxi Data

taxiSchema = (
                    StructType
                    ([ 
                        StructField("VendorId"               , IntegerType()   , True),
                        StructField("lpep_pickup_datetime"   , TimestampType() , True),
                        StructField("lpep_dropoff_datetime"  , TimestampType() , True),                            
                        StructField("passenger_count"        , DoubleType()    , True),
                        StructField("trip_distance"          , DoubleType()    , True),
                        StructField("RatecodeID"             , DoubleType()    , True),                            
                        StructField("store_and_fwd_flag"     , StringType()    , True),
                        StructField("PULocationID"           , IntegerType()   , True),
                        StructField("DOLocationID"           , IntegerType()   , True),                            
                        StructField("payment_type"           , IntegerType()   , True),                            
                        StructField("fare_amount"            , DoubleType()    , True),
                        StructField("extra"                  , DoubleType()    , True),
                        StructField("mta_tax"                , DoubleType()    , True),
                        StructField("tip_amount"             , DoubleType()    , True),
                        StructField("tolls_amount"           , DoubleType()    , True),
                        StructField("improvement_surcharge"  , DoubleType()    , True),
                        StructField("total_amount"           , DoubleType()    , True),
                        StructField("congestion_surcharge"   , DoubleType()    , True),
                        StructField("airport_fee"            , DoubleType()    , True)
                    ])
               )

In [5]:
# Read YellowTaxis CSV file

yellowTaxiDF = (
                  spark
                    .read
                    .option("header", "true")
    
                    .schema(taxiSchema)
    
                    .csv("C:\SparkCourse\DataFiles\Raw\YellowTaxis_202210.csv")
               )

yellowTaxiDF.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



### Create a SQL View based on Python DataFrame

In [6]:
yellowTaxiDF.createOrReplaceTempView("YellowTaxis")

### Run SQL query on View

In [7]:
outputDF = spark.sql(
                        "SELECT * FROM YellowTaxis WHERE PULocationID = 171"
                    )
    
outputDF.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-10-01 13:47:23|  2022-10-01 14:38:50|            1.0|          9.4|      99.0|                 N|         171|         263|           1|       35.2|  0.0|    0.5|       0.

### Read Green Taxis data, and create Temp View

In [8]:
# Read GreenTaxis TSV file

greenTaxiDF = (
                  spark
                    .read                     
                    .option("header", "true")    
                    .option("delimiter", "\t")    
                    .csv("C:\SparkCourse\DataFiles\Raw\GreenTaxis_202210.csv")
              )

greenTaxiDF.createOrReplaceTempView("GreenTaxis")

### Write SQL query to merge Yellow and Green Taxis data

In [9]:
spark.sql("""

SELECT 'Yellow'                   AS TaxiType

      , lpep_pickup_datetime      AS PickupTime
      , lpep_dropoff_datetime     AS DropTime
      , PULocationID              AS PickupLocationId
      , DOLocationID              AS DropLocationId      
FROM YellowTaxis

UNION ALL

SELECT 'Green'                    AS TaxiType

      , lpep_pickup_datetime      AS PickupTime
      , lpep_dropoff_datetime     AS DropTime
      , PULocationID              AS PickupLocationId
      , DOLocationID              AS DropLocationId 
FROM GreenTaxis

""").show()

+--------+-------------------+-------------------+----------------+--------------+
|TaxiType|         PickupTime|           DropTime|PickupLocationId|DropLocationId|
+--------+-------------------+-------------------+----------------+--------------+
|  Yellow|2022-10-01 05:33:41|2022-10-01 05:48:39|             249|           107|
|  Yellow|2022-10-01 05:44:30|2022-10-01 05:49:48|             151|           238|
|  Yellow|2022-10-01 05:57:13|2022-10-01 06:07:41|             238|           166|
|  Yellow|2022-10-01 06:02:53|2022-10-01 06:08:55|             142|           239|
|  Yellow|2022-10-01 06:14:55|2022-10-01 06:20:21|             238|           166|
|  Yellow|2022-10-01 05:52:52|2022-10-01 06:22:14|             186|            41|
|  Yellow|2022-10-01 06:03:19|2022-10-01 06:14:51|             162|           145|
|  Yellow|2022-10-01 05:32:42|2022-10-01 06:20:01|             100|            22|
|  Yellow|2022-10-01 05:36:35|2022-10-01 05:54:38|             138|           112|
|  Y

### Read Taxi Zones data, and create Global Temp View

In [10]:
taxiZonesSchema = "LocationID INT, Borough STRING, Zone STRING, ServiceZone STRING"

taxiZonesDF = (
                  spark
                    .read                    
                    .schema(taxiZonesSchema)
                    .csv("D:\DemoFiles\SparkCourseFiles\TaxiZones.csv")
              )


# Create global temp view
taxiZonesDF.createOrReplaceGlobalTempView("TaxiZones")


taxiZonesDF.show()

+----------+-------------+--------------------+-----------+
|LocationID|      Borough|                Zone|ServiceZone|
+----------+-------------+--------------------+-----------+
|         1|          EWR|      Newark Airport|        EWR|
|         2|       Queens|         Jamaica Bay|  Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|  Boro Zone|
|         4|    Manhattan|       Alphabet City|Yellow Zone|
|         5|Staten Island|       Arden Heights|  Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|  Boro Zone|
|         7|       Queens|             Astoria|  Boro Zone|
|         8|       Queens|        Astoria Park|  Boro Zone|
|         9|       Queens|          Auburndale|  Boro Zone|
|        10|       Queens|        Baisley Park|  Boro Zone|
|        11|     Brooklyn|          Bath Beach|  Boro Zone|
|        12|    Manhattan|        Battery Park|Yellow Zone|
|        13|    Manhattan|   Battery Park City|Yellow Zone|
|        14|     Brooklyn|           Bay

### Create a report 

Figure out number of rides, grouped by Borough and type of Taxi

In [11]:
spark.sql("""

SELECT Borough, TaxiType, COUNT(*) AS TotalTrips

FROM global_temp.TaxiZones

LEFT JOIN
(

    SELECT 'Yellow' AS TaxiType, PULocationID FROM YellowTaxis
    
    UNION ALL
    
    SELECT 'Green' AS TaxiType, PULocationID FROM GreenTaxis
    
) AllTaxis

ON AllTaxis.PULocationID = TaxiZones.LocationID

GROUP BY Borough, TaxiType

ORDER BY Borough, TaxiType

""").show()

+-------------+--------+----------+
|      Borough|TaxiType|TotalTrips|
+-------------+--------+----------+
|        Bronx|   Green|      1852|
|        Bronx|  Yellow|      4511|
|     Brooklyn|   Green|     11113|
|     Brooklyn|  Yellow|     28089|
|          EWR|   Green|        15|
|          EWR|  Yellow|      1157|
|    Manhattan|    null|         2|
|    Manhattan|   Green|     40545|
|    Manhattan|  Yellow|   3250695|
|       Queens|    null|         1|
|       Queens|   Green|     15377|
|       Queens|  Yellow|    333922|
|Staten Island|    null|         2|
|Staten Island|   Green|         8|
|Staten Island|  Yellow|       303|
|      Unknown|   Green|       412|
|      Unknown|  Yellow|     56735|
+-------------+--------+----------+

